# Clean KS301EW_Health.xlsx

In [11]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Reload the original cleaned percentage and numbers tables
file_health_2011_new = r"D:\Users\lenovo\Desktop\VA_Data\KS301EW_Health.xlsx"
df_pct = pd.read_excel(file_health_2011_new, sheet_name="KS301EW_Percentages", header=10)
print(df_pct.columns.tolist())


['Area code', 'Area name', 'Unnamed: 2', 'Unnamed: 3', 'All categories: Long-term health problem or disability', 'Day-to-day activities limited a lot', 'Day-to-day activities limited a little', 'Day-to-day activities not limited', 'Day-to-day activities limited a lot: Age 16 to 64', 'Day-to-day activities limited a little: Age 16 to 64', 'Day-to-day activities not limited: Age 16 to 64', 'Very good health', 'Good health', 'Fair health', 'Bad health', 'Very bad health', 'Provides no unpaid care', 'Provides 1 to 19 hours unpaid care a week', 'Provides 20 to 49 hours unpaid care a week', 'Provides 50 or more hours unpaid care a week']


In [15]:
# Re-import required libraries after reset
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Reload the original cleaned percentage and numbers tables
file_health_2011_new = r"D:\Users\lenovo\Desktop\VA_Data\KS301EW_Health.xlsx"
df_pct = pd.read_excel(file_health_2011_new, sheet_name="KS301EW_Percentages", header = 10)
df_numbers = pd.read_excel(file_health_2011_new, sheet_name="KS301EW_Numbers", header = 10)

# Rename geographic columns
geo_cols = {
    'Area name': 'Region',
    'Unnamed: 2': 'County',
    'Unnamed: 3': 'District'
}
df_pct.rename(columns=geo_cols, inplace=True)
df_numbers.rename(columns=geo_cols, inplace=True)

# Drop invalid rows and fill down geography
df_pct = df_pct[df_pct["Area code"].notna()].copy()
df_pct[["Region", "County", "District"]] = df_pct[["Region", "County", "District"]].fillna(method='ffill')

df_numbers = df_numbers[df_numbers["Area code"].notna()].copy()
df_numbers[["Region", "County", "District"]] = df_numbers[["Region", "County", "District"]].fillna(method='ffill')

# Rename health columns
df_pct = df_pct.rename(columns={
    "Very good health": "very_good",
    "Good health": "good",
    "Fair health": "fair",
    "Bad health": "bad",
    "Very bad health": "very_bad",
    "All categories: Long-term health problem or disability": "All_num"
})

df_numbers = df_numbers.rename(columns={
    "Very good health": "very_good_num",
    "Good health": "good_num",
    "Fair health": "fair_num",
    "Bad health": "bad_num",
    "Very bad health": "very_bad_num"
})

# Drop unrelated columns from percentage table
drop_keywords = ["Day-to-day", "Provides"]
cols_to_drop = [col for col in df_pct.columns if any(k in col for k in drop_keywords)]
df_pct.drop(columns=cols_to_drop, inplace=True)

# Compute health index and normalization
df_pct["health_index"] = (
    2 * df_pct["very_good"] +
    1 * df_pct["good"] +
    0 * df_pct["fair"] +
    (-1) * df_pct["bad"] +
    (-2) * df_pct["very_bad"]
)

scaler = MinMaxScaler(feature_range=(0, 100))
df_pct["health_index_norm"] = scaler.fit_transform(df_pct[["health_index"]])

# Merge only on Area code to ensure completeness
df_301 = pd.merge(df_pct, df_numbers[["Area code", "very_good_num", "good_num", "fair_num", "bad_num", "very_bad_num"]],
                     on="Area code", how="left")
df_301 = df_301[~df_301["Area code"].astype(str).str.contains("Source", na=False)].copy()
df_301.to_csv(r"D:\Users\lenovo\Desktop\VA_Data\301_cleaned.csv", index=False)
df_301


D:\Users\TEMP\ipykernel_8724\4253808378.py:21: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_pct[["Region", "County", "District"]] = df_pct[["Region", "County", "District"]].fillna(method='ffill')
D:\Users\TEMP\ipykernel_8724\4253808378.py:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_numbers[["Region", "County", "District"]] = df_numbers[["Region", "County", "District"]].fillna(method='ffill')


,Area code,Region,County,District,All_num,very_good,good,fair,bad,very_bad,health_index,health_index_norm,very_good_num,good_num,fair_num,bad_num,very_bad_num
0,K04000001,ENGLAND AND WALES,NaN,NaN,56075912,47.1,34.1,13.2,4.3,1.3,121.4,50.753769,26434409,19094820,7401881,2428668,716134
1,E92000001,ENGLAND,NaN,NaN,53012456,47.2,34.2,13.1,4.2,1.2,122.0,52.261307,25005712,18141457,6954092,2250446,660749
2,E12000001,NORTH EAST,NaN,NaN,2596886,44,33.3,15.2,5.8,1.7,112.1,27.386935,1142170,866035,395243,149374,44064
3,E06000047,NORTH EAST,County Durham UA,NaN,513242,42.4,33.4,16.1,6.3,1.8,108.3,17.839196,217373,171564,82404,32568,9333
4,E06000005,NORTH EAST,Darlington UA,NaN,105564,44.6,35.2,14.3,4.5,1.4,117.1,39.949749,47046,37199,15116,4763,1440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,W06000018,WALES,Somerset,Caerphilly,178806,44.4,30.4,15.9,7.2,2.1,107.8,16.582915,79382,54272,28488,12844,3820
391,W06000019,WALES,Somerset,Blaenau Gwent,69814,41.9,30.6,16.8,8.2,2.5,101.2,0.000000,29269,21385,11696,5692,1772
392,W06000020,WALES,Somerset,Torfaen,91075,43.3,32.5,15.5,6.5,2.1,108.4,18.090452,39398,29619,14155,5965,1938
393,W06000021,WALES,Somerset,Monmouthshire,91323,46.4,33.7,14,4.6,1.4,119.1,44.974874,42365,30742,12800,4173,1243


Check the missing value

In [40]:
# Count the number of missing values in each column
missing_summary = df_merged.isnull().sum().reset_index()
missing_summary.columns = ["Column", "Missing_Count"]
missing_summary = missing_summary[missing_summary["Missing_Count"] > 0]

# Display the missing fields and their quantities
missing_summary


,Column,Missing_Count
2,County,3
3,District,11


# Clean the KS401EW.xlsx

In [43]:
import pandas as pd

# 1. Read the Excel file
file_ks401 = r"D:\Users\lenovo\Desktop\VA_Data\KS401EW.xlsx"  

# 2. Read the percentage and number worksheets
df_pct = pd.read_excel(file_ks401, sheet_name="KS401EW_Percentages", header=10)
df_num = pd.read_excel(file_ks401, sheet_name="KS401EW_Numbers", header=10)

# 3. Geographical column renaming + filling
geo_cols = {
    'Area name': 'Region',
    'Unnamed: 2': 'County',
    'Unnamed: 3': 'District'
}
df_pct.rename(columns=geo_cols, inplace=True)
df_num.rename(columns=geo_cols, inplace=True)

df_pct = df_pct[df_pct["Area code"].notna()].copy()
df_pct[["Region", "County", "District"]] = df_pct[["Region", "County", "District"]].fillna(method='ffill')

df_num = df_num[df_num["Area code"].notna()].copy()
df_num[["Region", "County", "District"]] = df_num[["Region", "County", "District"]].fillna(method='ffill')

# 4. Define the columns to be retained and renamed
column_mapping_pct = {
    "Whole house or bungalow: Detached": "pct_detached",
    "Whole house or bungalow: Semi-detached": "pct_semi",
    "Whole house or bungalow: Terraced (including end-terrace)": "pct_terraced",
    "Flat, maisonette or apartment: Purpose-built block of flats or tenement": "pct_flat_built",
    "Flat, maisonette or apartment: Part of a converted or shared house (including bed-sits)": "pct_flat_converted",
    "Flat, maisonette or apartment: In a commercial building": "pct_flat_commercial",
    "Caravan or other mobile or temporary structure": "pct_mobile_home",
    "Household spaces with at least one usual resident": "pct_occupied"
}
column_mapping_num = {
    k: v.replace("pct_", "num_") for k, v in column_mapping_pct.items()
}
geo_keys = ["Area code", "Region", "County", "District"]

# 5. Filter and rename the variables
df_pct_clean = df_pct[geo_keys + list(column_mapping_pct.keys())].rename(columns=column_mapping_pct)
df_num_clean = df_num[geo_keys + list(column_mapping_num.keys())].rename(columns=column_mapping_num)

# 6.Merge the percentage and quantity table (by Area code)
df_401 = pd.merge(df_pct_clean, df_num_clean, on="Area code", how="left")

# 7. Delete non-data description rows (such as source)
df_401 = df_401[~df_401["Area code"].astype(str).str.contains("Source", na=False)].copy()

# 8. Remove repeated geographic fields (generated after merging the right table)
df_401.drop(columns=["Region_y", "County_y", "District_y"], errors="ignore", inplace=True)
df_401.rename(columns={"Region_x": "Region", "County_x": "County", "District_x": "District"}, inplace=True)

# save results
df_401.to_csv(r"D:\Users\lenovo\Desktop\VA_Data\401_cleaned.csv", index=False)

df_401

D:\Users\TEMP\ipykernel_10128\961407112.py:20: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_pct[["Region", "County", "District"]] = df_pct[["Region", "County", "District"]].fillna(method='ffill')
D:\Users\TEMP\ipykernel_10128\961407112.py:23: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_num[["Region", "County", "District"]] = df_num[["Region", "County", "District"]].fillna(method='ffill')


,Area code,Region,County,District,pct_detached,pct_semi,pct_terraced,pct_flat_built,pct_flat_converted,pct_flat_commercial,pct_mobile_home,pct_occupied,num_detached,num_semi,num_terraced,num_flat_built,num_flat_converted,num_flat_commercial,num_mobile_home,num_occupied
0,K04000001,ENGLAND AND WALES,NaN,NaN,22.6,30.7,24.7,16.3,4.2,1.1,0.4,95.6,5512289,7506350,6028348,3987968,1017335,271315,106013,23366044
1,E92000001,ENGLAND,NaN,NaN,22.3,30.7,24.5,16.7,4.3,1.1,0.4,95.7,5128552,7076395,5642969,3854451,984284,257218,100228,22063368
2,E12000001,NORTH EAST,NaN,NaN,15.7,38.7,30.4,12,2.3,0.8,0.1,95.8,184614,456002,358783,141829,27649,9044,1273,1129935
3,E06000047,NORTH EAST,County Durham UA,NaN,19,36.5,38.4,4.6,0.7,0.7,0.1,95.8,44287,85322,89782,10709,1569,1548,320,223803
4,E06000005,NORTH EAST,Darlington UA,NaN,17.2,37.5,31.2,11.1,2.2,0.8,0.1,95.7,8407,18283,15198,5411,1057,370,35,46670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,W06000018,WALES,Somerset,Caerphilly,16.1,39.2,36.1,6.9,0.8,0.9,0,96.4,12455,30270,27852,5320,627,686,34,74479
391,W06000019,WALES,Somerset,Blaenau Gwent,11,26.7,52.2,8,0.9,1.2,0.1,95.2,3506,8517,16669,2551,298,385,26,30416
392,W06000020,WALES,Somerset,Torfaen,17.8,26.9,42.9,10.8,0.8,0.7,0.2,95.8,7168,10806,17246,4349,316,273,71,38524
393,W06000021,WALES,Somerset,Monmouthshire,44,28.3,16.9,7.9,1.4,1.1,0.3,95.4,17638,11354,6789,3174,579,433,111,38233


Check the missing value

In [46]:
missing_ks401 = df_401.isnull().sum().reset_index()
missing_ks401.columns = ["Column", "Missing_Count"]
missing_ks401 = missing_ks401[missing_ks401["Missing_Count"] > 0]
missing_ks401

,Column,Missing_Count
2,County,3
3,District,11


# Clean the KS403EW.xlsx

In [49]:
import pandas as pd

# 1. Load the Excel file
file_ks403 = r"D:\Users\lenovo\Desktop\VA_Data\KS403EW.xlsx"

# 2. Load the percentage and quantity worksheet (using column names in Row 11)
df_pct = pd.read_excel(file_ks403, sheet_name="KS403EW_Percentages", header=10)
df_num = pd.read_excel(file_ks403, sheet_name="KS403EW_Numbers", header=10)

# 3. Rename the geographic field and fill it
geo_cols = {
    'Area name': 'Region',
    'Unnamed: 2': 'County',
    'Unnamed: 3': 'District'
}
df_pct.rename(columns=geo_cols, inplace=True)
df_num.rename(columns=geo_cols, inplace=True)

df_pct = df_pct[df_pct["Area code"].notna()].copy()
df_num = df_num[df_num["Area code"].notna()].copy()

df_pct[["Region", "County", "District"]] = df_pct[["Region", "County", "District"]].fillna(method='ffill')
df_num[["Region", "County", "District"]] = df_num[["Region", "County", "District"]].fillna(method='ffill')

# 4. Select and simplify the variable names
column_mapping_pct = {
    "Does have central heating": "pct_heated",
    "Occupancy rating (rooms) of -1 or less": "pct_overcrowded_rooms",
    "Occupancy rating (bedrooms) of -1 or less": "pct_overcrowded_beds",
    "Average household size": "avg_household_size",
    "Average number of rooms per household": "avg_rooms",
    "Average number of bedrooms per household": "avg_bedrooms"
}
column_mapping_num = {
    "Does have central heating": "num_heated",
    "Occupancy rating (rooms) of -1 or less": "num_overcrowded_rooms",
    "Occupancy rating (bedrooms) of -1 or less": "num_overcrowded_beds"
}
geo_keys = ["Area code", "Region", "County", "District"]

# 5. Filter and rename
df_pct_clean = df_pct[geo_keys + list(column_mapping_pct.keys())].rename(columns=column_mapping_pct)
df_num_clean = df_num[geo_keys + list(column_mapping_num.keys())].rename(columns=column_mapping_num)

# 6. Merge the percentage and quantity tables
df_403 = pd.merge(df_pct_clean, df_num_clean, on="Area code", how="left")

# 7. Delete explanatory non-data rows (including key phrases)
df_403 = df_403[~df_403["Area code"].astype(str).str.contains("bedroom standard|Source", na=False)]

# 8. Clean up duplicate geographic fields (*_x / *_y may appear after merging)
df_403.drop(columns=["Region_y", "County_y", "District_y"], errors="ignore", inplace=True)
df_403.rename(columns={"Region_x": "Region", "County_x": "County", "District_x": "District"}, inplace=True)

# 9. （save to the local
df_403.to_csv(r"D:\Users\lenovo\Desktop\VA_Data\403_cleaned.csv", index=False)

df_403


D:\Users\TEMP\ipykernel_10128\2923438698.py:22: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_pct[["Region", "County", "District"]] = df_pct[["Region", "County", "District"]].fillna(method='ffill')
D:\Users\TEMP\ipykernel_10128\2923438698.py:23: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_num[["Region", "County", "District"]] = df_num[["Region", "County", "District"]].fillna(method='ffill')


,Area code,Region,County,District,pct_heated,pct_overcrowded_rooms,pct_overcrowded_beds,avg_household_size,avg_rooms,avg_bedrooms,num_heated,num_overcrowded_rooms,num_overcrowded_beds
0,K04000001,ENGLAND AND WALES,NaN,NaN,97.3,8.5,4.5,2.4,5.4,2.7,22741949,1995860,1062644
1,E92000001,ENGLAND,NaN,NaN,97.3,8.7,4.6,2.4,5.4,2.7,21468807,1928596,1024473
2,E12000001,NORTH EAST,NaN,NaN,98.8,5.1,2.9,2.3,5.3,2.7,1116016,57273,33269
3,E06000047,NORTH EAST,County Durham UA,NaN,99.2,3.6,2.3,2.2,5.4,2.7,221935,8117,5124
4,E06000005,NORTH EAST,Darlington UA,NaN,98.3,4.6,2.7,2.2,5.4,2.7,45856,2124,1268
...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,W06000018,WALES,Somerset,Caerphilly,99.2,4.4,3,2.4,5.5,2.8,73913,3258,2249
391,W06000019,WALES,Somerset,Blaenau Gwent,99.2,5.3,3.3,2.3,5.2,2.7,30180,1605,1017
392,W06000020,WALES,Somerset,Torfaen,99,4.5,3.1,2.3,5.5,2.8,38143,1739,1201
393,W06000021,WALES,Somerset,Monmouthshire,98.4,3.6,2,2.3,6.2,3.1,37616,1362,771


Check missing value

In [52]:
# Check for missing values in the final KS403EW merged dataset
missing_ks403 = df_403.isnull().sum().reset_index()
missing_ks403.columns = ["Column", "Missing_Count"]
missing_ks403 = missing_ks403[missing_ks403["Missing_Count"] > 0]
missing_ks403

,Column,Missing_Count
2,County,3
3,District,11


# Merge the three 2011 datasets

In [61]:
# Re-import necessary libraries after environment reset
import pandas as pd

# Reload the three cleaned datasets
file_301 = r"D:\Users\lenovo\Desktop\VA_Data\301_cleaned.csv"
file_401 = r"D:\Users\lenovo\Desktop\VA_Data\401_cleaned.csv"
file_403 = r"D:\Users\lenovo\Desktop\VA_Data\403_cleaned.csv"

df_301 = pd.read_csv(file_301)
df_401 = pd.read_csv(file_401)
df_403 = pd.read_csv(file_403)

# Merge 301 and 401 on Area code
merged_301_401 = pd.merge(df_301, df_401.drop(columns=["Region", "County", "District"]), on="Area code", how="inner")

# Merge with 403
df_full = pd.merge(merged_301_401, df_403.drop(columns=["Region", "County", "District"]), on="Area code", how="inner")

#Save the file
df_full.to_csv(r"D:\Users\lenovo\Desktop\VA_Data\2011_cleaned.csv", index=False)

# Show the final merged dataset
df_full


,Area code,Region,County,District,All_num,very_good,good,fair,bad,very_bad,...,num_occupied,pct_heated,pct_overcrowded_rooms,pct_overcrowded_beds,avg_household_size,avg_rooms,avg_bedrooms,num_heated,num_overcrowded_rooms,num_overcrowded_beds
0,K04000001,ENGLAND AND WALES,NaN,NaN,56075912,47.1,34.1,13.2,4.3,1.3,...,23366044,97.3,8.5,4.5,2.4,5.4,2.7,22741949,1995860,1062644
1,E92000001,ENGLAND,NaN,NaN,53012456,47.2,34.2,13.1,4.2,1.2,...,22063368,97.3,8.7,4.6,2.4,5.4,2.7,21468807,1928596,1024473
2,E12000001,NORTH EAST,NaN,NaN,2596886,44.0,33.3,15.2,5.8,1.7,...,1129935,98.8,5.1,2.9,2.3,5.3,2.7,1116016,57273,33269
3,E06000047,NORTH EAST,County Durham UA,NaN,513242,42.4,33.4,16.1,6.3,1.8,...,223803,99.2,3.6,2.3,2.2,5.4,2.7,221935,8117,5124
4,E06000005,NORTH EAST,Darlington UA,NaN,105564,44.6,35.2,14.3,4.5,1.4,...,46670,98.3,4.6,2.7,2.2,5.4,2.7,45856,2124,1268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,W06000018,WALES,Somerset,Caerphilly,178806,44.4,30.4,15.9,7.2,2.1,...,74479,99.2,4.4,3.0,2.4,5.5,2.8,73913,3258,2249
391,W06000019,WALES,Somerset,Blaenau Gwent,69814,41.9,30.6,16.8,8.2,2.5,...,30416,99.2,5.3,3.3,2.3,5.2,2.7,30180,1605,1017
392,W06000020,WALES,Somerset,Torfaen,91075,43.3,32.5,15.5,6.5,2.1,...,38524,99.0,4.5,3.1,2.3,5.5,2.8,38143,1739,1201
393,W06000021,WALES,Somerset,Monmouthshire,91323,46.4,33.7,14.0,4.6,1.4,...,38233,98.4,3.6,2.0,2.3,6.2,3.1,37616,1362,771


# Clean the 2021 health data

In [10]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import os

# File Path
data_dir = r"D:\Users\lenovo\Desktop\VA_Data\TS037_Health"
mapping_path = r"D:\Users\lenovo\Desktop\VA_Data\Local_Authority_District_(2011)_to_Local_Authority_District_(2021)_Lookup_for_England_and_Wales.csv"
df_mapping = pd.read_csv(mapping_path)

# General cleaning function (Applicable to any geographical level)
def clean_health_data(filepath, map_lad11=False, mapping_df=None, scaler=None):
    df = pd.read_csv(filepath)

    df = df.rename(columns={
        "geography code": "Area code",
        "General health: Total: All usual residents": "All_num",
        "General health: Very good health": "very_good_num",
        "General health: Good health": "good_num",
        "General health: Fair health": "fair_num",
        "General health: Bad health": "bad_num",
        "General health: Very bad health": "very_bad_num"
    })

    for col in ["very_good", "good", "fair", "bad", "very_bad"]:
        df[col] = df[f"{col}_num"] / df["All_num"]

    df["health_index"] = (
        2 * df["very_good"] +
        1 * df["good"] +
        0 * df["fair"] +
        (-1) * df["bad"] +
        (-2) * df["very_bad"]
    )

    if scaler:
        df["health_index_norm"] = scaler.fit_transform(df[["health_index"]])
    else:
        df["health_index_norm"] = df["health_index"]

    if map_lad11 and mapping_df is not None:
        df = df.merge(mapping_df[["LAD21CD", "LAD11CD"]], left_on="Area code", right_on="LAD21CD", how="left")
        df["Area code"] = df["LAD11CD"]

    return df[[
        "Area code", "All_num",
        "very_good_num", "good_num", "fair_num", "bad_num", "very_bad_num",
        "very_good", "good", "fair", "bad", "very_bad",
        "health_index", "health_index_norm"
    ]]

# Initialize the unified normalizer
scaler = MinMaxScaler(feature_range=(0, 100))

# Clean four levels
df_ltla = clean_health_data(os.path.join(data_dir, "census2021-ts037-ltla.csv"), map_lad11=True, mapping_df=df_mapping, scaler=scaler)
df_utla = clean_health_data(os.path.join(data_dir, "census2021-ts037-utla.csv"), scaler=scaler)
df_rgn  = clean_health_data(os.path.join(data_dir, "census2021-ts037-rgn.csv"), scaler=scaler)
df_ctry = clean_health_data(os.path.join(data_dir, "census2021-ts037-ctry.csv"), scaler=scaler)

# Add levels source tags
df_ltla["source"] = "LTLA"
df_utla["source"] = "UTLA"
df_rgn["source"]  = "RGN"
df_ctry["source"] = "CTRY"

# Merge the four levels of data
df_all_levels = pd.concat([df_ltla, df_utla, df_rgn, df_ctry], ignore_index=True)

# Get all the area code(you should prepare this file first) from 2011 data.
df_2011 = pd.read_csv(r"D:\Users\lenovo\Desktop\VA_Data\2011_cleaned.csv")
area_codes_2011 = df_2011[["Area code"]].drop_duplicates()

# Find the optimal source of 2021 health data for each 2011 region (LTLA > UTLA > RGN > CTRY)
priority_order = ["LTLA", "UTLA", "RGN", "CTRY"]
merged_rows = []

for code in area_codes_2011["Area code"]:
    matched = df_all_levels[df_all_levels["Area code"] == code]
    if not matched.empty:
        for level in priority_order:
            subset = matched[matched["source"] == level]
            if not subset.empty:
                merged_rows.append(subset.iloc[0])
                break

# Build the final aligned DataFrame
df_2021_aligned = pd.DataFrame(merged_rows)

# Save to local
df_2021_aligned.to_csv(r"D:\Users\lenovo\Desktop\VA_Data\2021_aligned_health_data.csv", index=False)
print("Save as '2021_aligned_health_data.csv'")
df_2021_aligned

Save as '2021_aligned_health_data.csv'


,Area code,All_num,very_good_num,good_num,fair_num,bad_num,very_bad_num,very_good,good,fair,bad,very_bad,health_index,health_index_norm,source
533,K04000001,59597542,28827308,20046220,7597001,2412358,714655,0.483700,0.336360,0.127472,0.040477,0.011991,1.239299,94.785821,CTRY
532,E92000001,56490046,27390829,19040735,7147346,2248255,662881,0.484879,0.337064,0.126524,0.039799,0.011734,1.243553,100.000000,CTRY
522,E12000001,2647014,1180035,885603,395694,143103,42579,0.445799,0.334567,0.149487,0.054062,0.016086,1.139931,0.000000,RGN
44,E06000047,522068,224293,176980,81624,30167,9004,0.429624,0.338998,0.156347,0.057784,0.017247,1.105969,20.742109,LTLA
4,E06000005,107800,48749,37324,15383,5016,1328,0.452217,0.346234,0.142699,0.046531,0.012319,1.179499,37.455721,LTLA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,W06000018,175952,76930,56471,27645,11275,3631,0.437222,0.320945,0.157117,0.064080,0.020636,1.090036,17.120484,LTLA
342,W06000019,66904,27465,21865,11210,4785,1579,0.410514,0.326812,0.167554,0.071520,0.023601,1.029116,3.273286,LTLA
343,W06000020,92276,39335,31301,14340,5584,1716,0.426276,0.339211,0.155403,0.060514,0.018596,1.094055,18.033985,LTLA
344,W06000021,92955,43165,31887,12655,4060,1188,0.464364,0.343037,0.136141,0.043677,0.012780,1.202528,42.690288,LTLA


In [12]:
missing_2021 = df_2021_aligned.isnull().sum().reset_index()
missing_2021

,index,0
0,Area code,0
1,All_num,0
2,very_good_num,0
3,good_num,0
4,fair_num,0
5,bad_num,0
6,very_bad_num,0
7,very_good,0
8,good,0
9,fair,0


In [14]:
# Load the 2011 data
df_2011 = pd.read_csv(r"D:\Users\lenovo\Desktop\VA_Data\2011_cleaned.csv")

# Select the 2011 field for merging (only retain the fields related to health)
health_cols_2011 = [
    "Area code", "All_num", "very_good_num", "good_num", "fair_num", "bad_num", "very_bad_num",
    "very_good", "good", "fair", "bad", "very_bad", "health_index", "health_index_norm"
]
df_2011_health = df_2011[health_cols_2011].copy()
df_2011_health.columns = [col + "_2011" if col != "Area code" else col for col in df_2011_health.columns]

# The 2021 field was renamed to distinguish
df_2021_aligned_renamed = df_2021_aligned.copy()
df_2021_aligned_renamed = df_2021_aligned_renamed.drop(columns=["source"])
df_2021_aligned_renamed.columns = [col + "_2021" if col != "Area code" else col for col in df_2021_aligned_renamed.columns]

# Merge the health data of 2011 and 2021
df_comparison = df_2011_health.merge(df_2021_aligned_renamed, on="Area code", how="left")

# Save file
df_comparison.to_csv(r"D:\Users\lenovo\Desktop\VA_Data\2021_2011_comparison_health_data.csv", index=False)
print("Save as 2021_2011_comparison_health_data.csv'")
df_comparison

Save as 2021_2011_comparison_health_data.csv'


,Area code,All_num_2011,very_good_num_2011,good_num_2011,fair_num_2011,bad_num_2011,very_bad_num_2011,very_good_2011,good_2011,fair_2011,...,fair_num_2021,bad_num_2021,very_bad_num_2021,very_good_2021,good_2021,fair_2021,bad_2021,very_bad_2021,health_index_2021,health_index_norm_2021
0,K04000001,56075912,26434409,19094820,7401881,2428668,716134,47.1,34.1,13.2,...,7597001.0,2412358.0,714655.0,0.483700,0.336360,0.127472,0.040477,0.011991,1.239299,94.785821
1,E92000001,53012456,25005712,18141457,6954092,2250446,660749,47.2,34.2,13.1,...,7147346.0,2248255.0,662881.0,0.484879,0.337064,0.126524,0.039799,0.011734,1.243553,100.000000
2,E12000001,2596886,1142170,866035,395243,149374,44064,44.0,33.3,15.2,...,395694.0,143103.0,42579.0,0.445799,0.334567,0.149487,0.054062,0.016086,1.139931,0.000000
3,E06000047,513242,217373,171564,82404,32568,9333,42.4,33.4,16.1,...,81624.0,30167.0,9004.0,0.429624,0.338998,0.156347,0.057784,0.017247,1.105969,20.742109
4,E06000005,105564,47046,37199,15116,4763,1440,44.6,35.2,14.3,...,15383.0,5016.0,1328.0,0.452217,0.346234,0.142699,0.046531,0.012319,1.179499,37.455721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,W06000018,178806,79382,54272,28488,12844,3820,44.4,30.4,15.9,...,27645.0,11275.0,3631.0,0.437222,0.320945,0.157117,0.064080,0.020636,1.090036,17.120484
391,W06000019,69814,29269,21385,11696,5692,1772,41.9,30.6,16.8,...,11210.0,4785.0,1579.0,0.410514,0.326812,0.167554,0.071520,0.023601,1.029116,3.273286
392,W06000020,91075,39398,29619,14155,5965,1938,43.3,32.5,15.5,...,14340.0,5584.0,1716.0,0.426276,0.339211,0.155403,0.060514,0.018596,1.094055,18.033985
393,W06000021,91323,42365,30742,12800,4173,1243,46.4,33.7,14.0,...,12655.0,4060.0,1188.0,0.464364,0.343037,0.136141,0.043677,0.012780,1.202528,42.690288


In [16]:
missing_comparison = df_comparison.isnull().sum().reset_index()
missing_comparison

,index,0
0,Area code,0
1,All_num_2011,0
2,very_good_num_2011,0
3,good_num_2011,0
4,fair_num_2011,0
5,bad_num_2011,0
6,very_bad_num_2011,0
7,very_good_2011,0
8,good_2011,0
9,fair_2011,0


Use Bayes method to handle missing value

In [35]:
import pandas as pd
from sklearn.linear_model import BayesianRidge

# Reload the original comparison file with missing values for proper handling
original_path = r"D:\Users\lenovo\Desktop\VA_Data\2021_2011_comparison_health_data.csv"
df_original = pd.read_csv(original_path)

# Identify rows with and without missing 2021 values
missing_rows = df_original[df_original["All_num_2021"].isnull()].copy()
non_missing_rows = df_original.dropna(subset=["All_num_2021"]).copy()

# Predictors from 2011 used for Bayesian regression
predictors = [
    "very_good_2011", "good_2011", "fair_2011",
    "bad_2011", "very_bad_2011",
    "health_index_2011", "health_index_norm_2011"
]

# Targets to impute
targets = [col for col in df_original.columns if col.endswith("_2021")]

# Container for predictions
from sklearn.linear_model import BayesianRidge
imputed_data = missing_rows[["Area code"]].copy()

# Build and apply model per target
for target in targets:
    model = BayesianRidge()
    model.fit(non_missing_rows[predictors], non_missing_rows[target])
    imputed_data[target] = model.predict(missing_rows[predictors])

# Merge back into the full dataset, ensuring proper alignment and overwrite
df_repaired = df_original.copy()
df_repaired = df_repaired.drop(columns=targets)
df_repaired = df_repaired.merge(imputed_data, on="Area code", how="left", suffixes=("", "_imputed"))

# Overwrite or fill with predictions
for target in targets:
    df_repaired[target] = df_repaired[target].combine_first(df_original[target])

# Final check: missing values
final_missing = df_repaired.isnull().sum().sum()

# Save results
df_repaired.to_csv(r"D:\Users\lenovo\Desktop\VA_Data\2021_2011_health_filled_fixed.csv", index=False)
print("Bayes filling is complete and the result is saved as '2021_2011_health_filled_fixed.csv'")


Bayes filling is complete and the result is saved as '2021_2011_health_filled_fixed.csv'


In [37]:
final_missing

0